In [1]:
#!pip install datefinder

In [31]:
import pandas as pd
import numpy as np
import datefinder as dfi

In [23]:
################################################
#FUNCIONES NO IMPORTADAS: PASAR A .PY E IMPORTAR
################################################

def del_p(x):
    ''' This function which takes a string as argument, returns the same string
    that was entered but without (p) in case there are any of these in the original
    string
    '''
    try:
        return x.replace('(p)', '')
    except:
        pass

def del_years(x):
    ''' This function returns a nan if the given argument (must be a string)
    has 4 or less characters. Otherwise  it returns the intact original string.
    The original purpose of this function was to delete cells with year info but 
    no month info.
    '''
    import numpy as np
    if len(x.strip()) < 5:
        return 'tbd'
    else:
        return x
    
def type_el(x):
    '''Returns the type of an element'''
    print(type(x))
    
def del_spaces(x):
    ''' Function to apply the strip method for strings'''
    return x.strip()

def translate_month(x):
    if 'Enero' in x:
        return x.replace('Enero', 'January')
    elif 'Febrero' in x:
        return x.replace('Febrero', 'February')
    elif 'Marzo' in x:
        return x.replace('Marzo', 'March')
    elif 'Abril' in x:
        return x.replace('Abril', 'April')
    elif 'Mayo' in x:
        return x.replace('Mayo', 'May')
    elif 'Junio' in x:
        return x.replace('Junio', 'June')
    elif 'Julio' in x:
        return x.replace('Julio', 'July')
    elif 'Agosto' in x:
        return x.replace('Agosto', 'August')
    elif 'Septiembre' in x:
        return x.replace('Septiembre', 'September')
    elif 'Octubre' in x:
        return x.replace('Octubre', 'October')
    elif 'Noviembre' in x:
        return x.replace('Noviembre', 'November')
    elif 'Diciembre' in x:
        return x.replace('Diciembre', 'December')
    else:
        return x
    

In [28]:
IC_trade = pd.read_csv ('canary_imports_dirty.csv')

In [25]:
IC_trade.head(25)

,Unnamed: 0,Time,Polonia,Chequia,Suiza,Francia,Italia,Bélgica,Dinamarca,Suecia,Irlanda,Países Bajos,Reino Unido,Alemania
0,0,2021 Noviembre (p),"4.473,862","4.698,110","2.220,403","31.485,825","36.338,226","6.008,680","2.178,621","1.574,086","6.561,863","36.734,327","11.720,918","29.201,971"
1,1,2021 Octubre (p),"3.360,605","9.997,751","1.711,822","7.672,856","12.718,099","4.617,084","2.493,726","1.773,129","3.440,667","29.173,793","6.908,026","25.644,328"
2,2,2021 Septiembre (p),"2.619,986","7.827,948","1.331,296","11.305,727","10.842,908","6.359,599","1.133,544","1.540,708","5.547,320","39.631,925","6.700,103","26.015,663"
3,3,2021 Agosto (p),"2.627,623","3.586,898","1.031,052","7.893,321","13.490,680","11.802,737","1.636,533","1.318,356","5.301,273","16.572,673","4.030,151","39.025,092"
4,4,2021 Julio (p),"2.102,901","10.212,904","1.264,930","7.513,558","16.195,542","10.413,309","1.557,271","1.475,441","4.772,880","18.454,604","3.430,143","35.785,879"
5,5,2021 Junio (p),"3.388,428","10.449,205","1.160,268","9.134,338","14.147,101","13.254,148","1.643,764","2.083,160","4.013,956","16.306,432","5.285,405","34.832,457"
6,6,2021 Mayo (p),"3.363,464","10.226,112","1.643,662","13.386,345","15.175,172","11.772,811","1.771,438","1.355,619","3.692,641","36.470,209","5.585,533","41.705,798"
7,7,2021 Abril (p),"2.886,051","9.442,443","4.361,508","7.358,942","13.072,034","7.182,839","2.343,504","1.608,213","5.059,566","12.633,066","3.989,645","25.069,301"
8,8,2021 Marzo (p),"3.759,953","9.289,201","1.464,496","8.927,103","12.944,139","25.616,531","2.305,670","17.249,702","4.266,997","24.401,167","5.563,745","38.099,255"
9,9,2021 Febrero (p),"2.330,797","9.647,049","1.167,256","5.371,861","11.452,291","4.998,165","1.601,125","789,361","1.739,589","10.629,715","4.634,569","23.511,202"


In [26]:
#Displaying unique values of the Time column in order to clean it

#IC_trade['Time'].unique()

In [29]:
################################################
#OJO - función no importada en uso en esta celda
################################################

IC_trade.Time = IC_trade.Time.apply(del_years)
IC_trade.Time = IC_trade.Time.apply(del_p)
IC_trade.Time = IC_trade.Time.apply(del_years)
IC_trade.Time = IC_trade.Time.apply(del_spaces)
IC_trade.Time = IC_trade.Time.apply(translate_month)
print(IC_trade['Time'].unique())


['2021 November' '2021 October' '2021 September' '2021 August' '2021 July'
 '2021 June' '2021 May' '2021 April' '2021 March' '2021 February'
 '2021 January' 'tbd' '2020 December' '2020 November' '2020 October'
 '2020 September' '2020 August' '2020 July' '2020 June' '2020 May'
 '2020 April' '2020 March' '2020 February' '2020 January' '2019 December'
 '2019 November' '2019 October' '2019 September' '2019 August' '2019 July'
 '2019 June' '2019 May' '2019 April' '2019 March' '2019 February'
 '2019 January' '2018 December' '2018 November' '2018 October'
 '2018 September' '2018 August' '2018 July' '2018 June' '2018 May'
 '2018 April' '2018 March' '2018 February' '2018 January' '2017 December'
 '2017 November' '2017 October' '2017 September' '2017 August' '2017 July'
 '2017 June' '2017 May' '2017 April' '2017 March' '2017 February'
 '2017 January' '2016 December' '2016 November' '2016 October'
 '2016 September' '2016 August' '2016 July' '2016 June' '2016 May'
 '2016 April' '2016 March' '2016 

In [21]:
################################################
#OJO - función no importada en uso en esta celda
################################################

In [37]:
matches = dfi.find_dates('2001 January')

In [38]:
for match in matches:
    print(match)

2001-01-05 00:00:00


# A. Trade flows (Canary)

# B. Trade flows (Balearic)

# C. Foreign citizens in the Canaries and the Balearic islands